# Testing the pipeline with Newsapi instead of GoogleNews





## Import stuff

In [1]:
!pip install datasets
!pip install transformers
!pip install huggingface_hub
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install torch
!pip install googlesearch-python
!pip install GoogleNews
!pip install textblob
!pip install schedule
!pip install time
!pip install telebot
!pip install telethon
!pip install transformers
!pip install vadersentiment
!pip install alpha_vantage
!pip install Prophet
!pip install tensorflow
!pip install optuna
!pip install --upgrade tensorflow
!pip install newsapi-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
     

In [2]:
import requests
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import schedule
import time
from transformers import pipeline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from GoogleNews import GoogleNews
import yfinance as yf
import torch
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

## Telegram Tokens

In [ ]:
# Include Telegram bot token and chat ID for pushing forecast results
BOT_TOKEN = "<BOT_TOKEN>"
chat_id = "<CHAT_ID>" 

# Data

## Numeric

In [3]:
def get_numeric_data(ticker):
    # Define the ticker symbol and time period
    period = "1mo" #must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']"
    interval = "1h"

    # Retrieve the historical market data
    data = yf.download(tickers=ticker, period=period, interval=interval)

    # Retrieve the ticker info
    ticker_info = yf.Ticker(ticker).info

    # Extract the market cap
    #market_cap = ticker_info['marketCap']

    # Calculate the moving averages
    data['MA_50'] = data['Close'].rolling(window=50).mean()
    data['MA_200'] = data['Close'].rolling(window=200).mean()

    # Calculate the RSI
    delta = data['Close'].diff(1)
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    roll_up = up.rolling(window=14).mean()
    roll_down = down.rolling(window=14).mean().abs()
    RS = roll_up / roll_down
    data['RSI'] = 100.0 - (100.0 / (1.0 + RS))

    # Calculate the MACD
    data['EMA_12'] = data['Close'].ewm(span=12, adjust=False).mean()
    data['EMA_26'] = data['Close'].ewm(span=26, adjust=False).mean()
    data['MACD'] = data['EMA_12'] - data['EMA_26']
    data['Signal'] = data['MACD'].ewm(span=9, adjust=False).mean()

    # Create a single dataframe called numeric_df
    numeric_df = data.select_dtypes(include=['number'])

    # Reset the index
    numeric_df.reset_index(inplace=True)

    return numeric_df

#ticker = 'SPY'
#numeric = get_numeric_data(ticker)
#numeric.head()

## Sentiment

### Get Data

In [ ]:
import os
import pandas as pd
from newsapi import NewsApiClient
from datetime import datetime, timedelta
def get_news():
    # Set API key and parameters
    NEWS_API_KEY = "<NEWS_API_TOKEN>"
    QUERY = "stock market"  # or any other query you're interested in
    START_DATE = "2024-06-30"
    END_DATE = "2024-07-30"
    LANGUAGE = "en"

    # Initialize NewsAPI client
    newsapi = NewsApiClient(api_key=NEWS_API_KEY)

    # Define a function to fetch news articles
    def fetch_news(start_date, end_date):
        articles = []
        for page in range(1, 6):  # fetch up to 5 pages of results
            response = newsapi.get_everything(
                q=QUERY,
                language=LANGUAGE,
                from_param=start_date,
                to=end_date,
                page=page
            )
            articles.extend(response["articles"])
        return articles

    # Fetch news articles
    articles = fetch_news(START_DATE, END_DATE)

    # Convert articles to a Pandas DataFrame
    df = pd.DataFrame(articles)

    # Extract relevant columns
    df = df[["title", "description", "publishedAt"]]

    # Convert publishedAt to datetime
    df["publishedAt"] = pd.to_datetime(df["publishedAt"])

    return df
#df = get_news()
#df.tail()

In [ ]:
import os
import pandas as pd
from newsapi import NewsApiClient
from datetime import datetime, timedelta

def get_news():
    # Set API key and parameters
    NEWS_API_KEY = "<NEWS_API_TOKEN>"
    QUERY = "stock market"  # or any other query you're interested in
    START_DATE = "2024-07-01"
    END_DATE = "2024-07-29"
    LANGUAGE = "en"

    # Initialize NewsAPI client
    newsapi = NewsApiClient(api_key=NEWS_API_KEY)

    # Define a function to fetch news articles
    def fetch_news(start_date, end_date):
        articles = []
        for page in range(1, 6):  # fetch up to 5 pages of results
            response = newsapi.get_everything(
                q=QUERY,
                language=LANGUAGE,
                from_param=start_date,
                to=end_date,
                page=page
            )
            articles.extend(response["articles"])
        return articles

    # Fetch news articles
    articles = fetch_news(START_DATE, END_DATE)

    # Convert articles to a Pandas DataFrame
    df = pd.DataFrame(articles)

    # Extract relevant columns
    df = df[["title", "description", "publishedAt"]]

    # Convert publishedAt to datetime
    df["publishedAt"] = pd.to_datetime(df["publishedAt"])

    # Get today's date and the date two days ago
    today = datetime.now().strftime("%Y-%m-%d")
    two_days_ago = (datetime.now() - timedelta(days=2)).strftime("%Y-%m-%d")

    # Fetch news articles from the past two days
    recent_articles = fetch_news(two_days_ago, today)

    # Convert recent articles to a Pandas DataFrame
    recent_df = pd.DataFrame(recent_articles)

    # Extract relevant columns
    recent_df = recent_df[["title", "description", "publishedAt"]]

    # Convert publishedAt to datetime
    recent_df["publishedAt"] = pd.to_datetime(recent_df["publishedAt"])

    # Append recent articles to the existing dataframe
    df = pd.concat([df, recent_df], ignore_index=True)

    return df

df = get_news()
df.tail()

,title,description,publishedAt
995,Hookipa Pharma (NASDAQ:HOOK) Price Target Lowe...,Hookipa Pharma (NASDAQ:HOOK – Get Free Report)...,2024-07-28 10:50:42+00:00
996,Leslie’s (NASDAQ:LESL) Shares Gap Up to $2.86,"Leslie’s, Inc. (NASDAQ:LESL – Get Free Report)...",2024-07-28 07:34:43+00:00
997,Loews (NYSE:L) Upgraded by Royal Bank of Canad...,Royal Bank of Canada upgraded shares of Loews ...,2024-07-27 11:31:16+00:00
998,Groupon (GRPN) to Release Earnings on Tuesday,Groupon (NASDAQ:GRPN – Get Free Report) is sch...,2024-07-28 07:56:43+00:00
999,Piedmont Lithium (NASDAQ:PLL) Given New $15.00...,Piedmont Lithium (NASDAQ:PLL – Get Free Report...,2024-07-28 08:50:44+00:00


In [26]:
df.head()

,title,description,publishedAt
0,Trump Stock Jumps Following Assassination Attempt,The meme stock undergirding Truth Social rose ...,2024-07-15 14:25:42+00:00
1,Stock market today: Dow closes at a record as ...,Investors betting on a second Trump presidency...,2024-07-15 20:09:18+00:00
2,Stock market today: Indexes rise as traders as...,"Consumer spending remained strong in June, red...",2024-07-16 13:41:45+00:00
3,A reliable stock market indicator is flashing ...,"""Credit markets are not confirming the equity ...",2024-07-16 16:58:25+00:00
4,Stock market today: Indexes test record highs ...,"The June jobs ""should have Fed officials troub...",2024-07-05 13:39:38+00:00


## Roberta

In [5]:
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

def roberta_sentiment_analysis(df):
    df = df.copy()

    # Load pre-trained RoBERTa model and tokenizer
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)

    # Preprocess headlines data
    encoded_headlines = df['title'].apply(lambda x: tokenizer.encode_plus(x,
                                                                          add_special_tokens=True,
                                                                          max_length=512,
                                                                          return_attention_mask=True,
                                                                          return_tensors='pt',
                                                                          truncation=True))

    # Extract the input_ids and attention_mask from the encoded headlines
    input_ids = encoded_headlines.apply(lambda x: x['input_ids'].flatten().tolist())
    attention_mask = encoded_headlines.apply(lambda x: x['attention_mask'].flatten().tolist())

    # Create new columns in the dataframe for input_ids and attention_mask
    df['input_ids'] = input_ids
    df['attention_mask'] = attention_mask

    # Create a list to store sentiment scores
    sentiment_scores = []

    # Perform sentiment analysis on each headline
    for input_id, attention_mask in zip(df['input_ids'], df['attention_mask']):
        # Create a tensor dataset and data loader
        input_id_tensor = torch.tensor([input_id])
        attention_mask_tensor = torch.tensor([attention_mask])
        dataset = torch.utils.data.TensorDataset(input_id_tensor, attention_mask_tensor)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=1)

        # Perform inference
        with torch.no_grad():
            for batch in data_loader:
                input_id, attention_mask = batch
                outputs = model(input_id, attention_mask=attention_mask)
                logits = outputs.logits
                sentiment_score = torch.softmax(logits, dim=1)[0][1].item()
                sentiment_scores.append(sentiment_score)

    # Normalize sentiment scores to be between 0 and 1
    min_score = min(sentiment_scores)
    max_score = max(sentiment_scores)
    normalized_sentiment_scores = [(score - min_score) / (max_score - min_score) for score in sentiment_scores]

    # Add normalized sentiment scores to the dataframe
    df['sentiment'] = normalized_sentiment_scores
    df.drop('attention_mask', axis=1, inplace=True)

    # Convert publishedAt to datetime and set as index
    df['publishedAt'] = pd.to_datetime(df['publishedAt'])
    df.set_index('publishedAt', inplace=True)

    # Group by hour and calculate mean sentiment score
    df_grouped = df['sentiment'].groupby(df.index.floor('H')).mean().reset_index()
    df_grouped.columns = ['hour', 'sentiment_score']

    return df_grouped

#df = get_news()
#sentiment_scores = roberta_sentiment_analysis(df)
#sentiment_scores.head()

## Vader

In [6]:
#VaderSentiment
def vader_sentiment_analysis(df):
    df = df.copy()
    # Create a SentimentIntensityAnalyzer object
    sia = SentimentIntensityAnalyzer()

    # Apply sentiment analysis to each headline
    df['sentiment'] = df['title'].apply(lambda x: sia.polarity_scores(x)['compound'])

    # Normalize sentiment scores to be between 0 and 1
    df['sentiment'] = df['sentiment'].apply(lambda x: (x + 1) / 2)

    # Convert publishedAt to datetime and set as index
    df['publishedAt'] = pd.to_datetime(df['publishedAt'])
    df.set_index('publishedAt', inplace=True)

    # Group by hour and calculate mean sentiment score
    df_grouped = df['sentiment'].groupby(df.index.floor('H')).mean().reset_index()
    df_grouped.columns = ['hour', 'sentiment_score']

    return df_grouped

#df = get_news()
#sentiment_scores = vader_sentiment_analysis(df)
#sentiment_scores.head()

## Blob

In [7]:
from textblob import TextBlob
import pandas as pd

def blob_sentiment_analysis(df):
    df = df.copy()
    # Create a new column to store the sentiment scores
    df['sentiment'] = df['title'].apply(lambda x: TextBlob(x).sentiment.polarity)

    # Normalize the sentiment scores to a range of 0 to 1
    min_sentiment = df['sentiment'].min()
    max_sentiment = df['sentiment'].max()
    df['sentiment_normalized'] = (df['sentiment'] - min_sentiment) / (max_sentiment - min_sentiment)

    # Convert publishedAt to datetime and set as index
    df['publishedAt'] = pd.to_datetime(df['publishedAt'])
    df.set_index('publishedAt', inplace=True)

    # Group by hour and calculate mean sentiment score
    df_grouped = df['sentiment_normalized'].groupby(df.index.floor('H')).mean().reset_index()
    df_grouped.columns = ['hour', 'sentiment_score']

    return df_grouped

#df = get_news()
#sentiment_scores = blob_sentiment_analysis(df)
#sentiment_scores.head()

## Ensemble

In [8]:
def ensemble_sentiment_analysis(df):
    # Perform sentiment analysis using each model
    roberta_scores = roberta_sentiment_analysis(df)
    vader_scores = vader_sentiment_analysis(df)
    textblob_scores = blob_sentiment_analysis(df)

    # Create a list to store the sentiment scores from each model
    sentiment_scores = [roberta_scores, vader_scores, textblob_scores]

    # Create a DataFrame to store the ensemble sentiment scores
    ensemble_df = pd.DataFrame({'hour': roberta_scores['hour']})

    # Calculate the average sentiment score across all models for each hour
    ensemble_df['ensemble_sentiment'] = np.mean([scores['sentiment_score'] for scores in sentiment_scores], axis=0)

    # Calculate the standard deviation of sentiment scores across all models for each hour
    ensemble_df['ensemble_sentiment_std'] = np.std([scores['sentiment_score'] for scores in sentiment_scores], axis=0)

    return ensemble_df

#df = get_news()
#sentiment_scores = ensemble_sentiment_analysis(df)
#sentiment_scores.head()

In [9]:
def sentiment_ensemble_pipeline():
  headlines = get_news()
  sentiments = ensemble_sentiment_analysis(headlines)
  return sentiments

## Concat Data

In [10]:
def concat_numeric_sentiments(numeric_df, sentiment_df):
    # Convert the "Datetime" column to datetime format
    numeric_df['Datetime'] = pd.to_datetime(numeric_df['Datetime'])

    # Extract the hour from the "Datetime" column
    numeric_df['hour'] = numeric_df['Datetime'].dt.floor('H')

    # Merge the two dataframes based on the hour
    combined_df = pd.merge(numeric_df, sentiment_df, left_on='hour', right_on='hour')

    # Drop the "hour" column
    combined_df = combined_df.drop('hour', axis=1)

    return combined_df

#ticker = 'SPY'
#numeric_df = get_numeric_data(ticker)
#sentiment_df = sentiment_ensemble_pipeline()
#combined_df = concat_numeric_sentiments(numeric_df, sentiment_df)
#combined_df.head()

# Data Pipeline

In [30]:
import pandas as pd

def gn_scrape_news_articles(ticker, num_articles):
    gn = GoogleNews(lang='en')
    gn.search(ticker)
    gn_news_articles = gn.results(num_articles)
    #gn_articles = [{'title': article['title'], 'date': article['date']} for article in gn_news_articles]
    df = pd.DataFrame(gn_news_articles)
    return df

ticker = 'SPY'
num_articles = 10
gn_titles = gn_scrape_news_articles(ticker, num_articles)
gn_titles

,title,media,date,datetime,desc,link,img
0,"Market Clubhouse Morning Memo - July 29th, 202...",Benzinga,3 hours ago,2024-07-29 14:41:01.501273,Good Morning Traders! In today's Market Clubho...,https://www.benzinga.com/24/07/40013656/market...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
1,"Weekly ETF flows: SPY bags $5.3B inflow, most ...",Seeking Alpha,3 hours ago,2024-07-29 14:41:01.479131,The SPDR S&P 500 Trust (SPY) saw a positive in...,https://seekingalpha.com/news/4129479-weekly-e...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
2,"SPY ETF Update, 7/29/2024",Tipranks,9 hours ago,2024-07-29 08:41:01.526526,How is SPY stock faring? The SPDR SP 500 ETF T...,https://www.tipranks.com/news/qqq-etf-update-7...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
3,EP Wealth Advisors LLC Reduces Stake in SPDR S...,Defense World,1 day ago,2024-07-28 17:41:01.569465,Read EP Wealth Advisors LLC Reduces Stake in S...,https://www.defenseworld.net/2024/07/28/ep-wea...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
4,(SPY) Trading Advice,Stock Traders Daily,1 day ago,2024-07-28 17:41:01.548611,Trading Advice for S&p 500 Etf Trust Etf SPY i...,https://news.stocktradersdaily.com/news_releas...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
5,S&P 500 ends wild week nearly 1% lower as tech...,Seeking Alpha,2 days ago,2024-07-27 17:41:01.613558,The S&P 500 (SP500) on Friday retreated 0.84% ...,https://seekingalpha.com/news/4129211-sp-500-e...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
6,Wall Street Exclusive - All About Earnings (NY...,Seeking Alpha,2 days ago,2024-07-27 17:41:01.591965,Fed Fund futures have priced in a 100% probabi...,https://seekingalpha.com/article/4707384-wall-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
7,"SPY ETF Update, 7/26/2024",Tipranks,3 days ago,2024-07-26 17:41:01.660448,How is SPY stock faring? The SPDR SP 500 ETF T...,https://www.tipranks.com/news/spy-etf-update-7...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
8,S&P 500 today: SPX is up 14.51% YTD,USA Today,3 days ago,2024-07-26 17:41:01.636430,The S&P 500 opened today at 5433.67. Within 30...,https://www.usatoday.com/money/blueprint/inves...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
9,Is It Time To Turn Intermediate Term Bearish O...,Seeking Alpha,4 days ago,2024-07-25 17:41:01.680712,The substantive correction in stock prices ant...,https://seekingalpha.com/article/4706559-is-it...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."


In [11]:
def numeric_sentiment_pipeline(ticker):
  numeric_df = get_numeric_data(ticker)
  sentiment_df = sentiment_ensemble_pipeline()
  combined_df = concat_numeric_sentiments(numeric_df, sentiment_df)
  return combined_df

#ticker = 'FNGU'
#df = numeric_sentiment_pipeline(ticker)
#df.tail()

In [24]:
df.tail()

,Datetime,Open,High,Low,Close,Adj Close,Volume,MA_50,MA_200,RSI,EMA_12,EMA_26,MACD,Signal,ensemble_sentiment,ensemble_sentiment_std
92,2024-07-25 15:30:00-04:00,394.869995,395.250000,376.790009,376.790009,376.790009,113576,436.360714,466.154937,18.451905,397.435657,415.026562,-17.590905,-16.557256,0.434500,0.046372
93,2024-07-26 09:30:00-04:00,389.769989,389.989990,377.184998,383.265106,383.265106,357454,434.006016,466.034312,33.468199,395.255572,412.673861,-17.418289,-16.729462,0.518409,0.143062
94,2024-07-26 10:30:00-04:00,382.970001,383.170013,376.000000,378.869995,378.869995,141160,432.375015,465.786212,35.608663,392.734714,410.169871,-17.435157,-16.870601,0.528874,0.064246
95,2024-07-26 11:30:00-04:00,379.424988,389.574005,378.429993,388.690002,388.690002,98332,430.958815,465.594313,45.470754,392.112451,408.578770,-16.466319,-16.789745,0.549718,0.243011
96,2024-07-26 13:30:00-04:00,393.279999,393.518799,383.214996,386.480011,386.480011,98830,428.514014,465.261193,46.843452,391.452570,405.921325,-14.468755,-16.058381,0.377719,0.099428


# GRU forcast

In [12]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping

def gru_forcast(target, df, future_days):
    print(target)
    num_mc_simulations=100
    # Load the data
    df = df.copy()

    # Convert the Datetime column to datetime format
    df['Datetime'] = pd.to_datetime(df['Datetime'])

    # Move the Datetime column to the first position
    cols = df.columns.tolist()
    cols.insert(0, cols.pop(cols.index('Datetime')))
    df = df[cols]

    # Create a new dataframe with only the 'Close' column
    close_df = df[[target]]

    # Scale the data
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_close = scaler.fit_transform(close_df)

    # Prepare the input data for the model
    seq_len = 1
    X_train = []
    y_train = []
    for i in range(seq_len, len(scaled_close)):
        X_train.append(scaled_close[i-seq_len:i])
        y_train.append(scaled_close[i])
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

    last_seq = scaled_close[-seq_len:]
    last_seq = last_seq.reshape((1, seq_len, 1))

    # Create the GRU model
    model = Sequential()
    model.add(GRU(units=1000, return_sequences=True, input_shape=(seq_len, 1)))
    model.add(Dropout(0.26))
    model.add(GRU(units=200, return_sequences=True))
    model.add(Dropout(0.26))
    model.add(GRU(units=1000, return_sequences=False))
    model.add(Dropout(0.26))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    early_stopping = EarlyStopping(monitor='loss', patience=5, min_delta=0.001)

    # Train the model
    model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=0, callbacks=[early_stopping])
    #model.save('gru_model_pred-for.keras')

    # Make predictions on known data
    predictions = model.predict(X_train)

    # Rescale the predictions
    predictions_rescaled = scaler.inverse_transform(predictions)

    # Calculate actual prices
    actual_prices = close_df[seq_len:].values

    # Calculate MAE
    mae = np.mean(np.abs(actual_prices - predictions_rescaled))
    print(f'MAE: {mae:.2f}')

    # Calculate RMSPE
    rmspe = np.sqrt(np.mean((actual_prices - predictions_rescaled) ** 2)) / np.mean(actual_prices) * 100
    print(f'RMSPE: {rmspe:.2f}%')

    # Plot actual and predicted prices
    plt.plot(close_df[seq_len:], label='Actual')
    plt.plot(predictions_rescaled, label='Predicted')
    plt.legend()
    plt.show()

    #future high
    future_close_prices_mc = []
    for _ in range(num_mc_simulations):
        future_close_prices = []
        last_seq_mc = scaled_close[-seq_len:]
        last_seq_mc = last_seq_mc.reshape((1, seq_len, 1))
        for i in range(future_days):
            predictions = model.predict(last_seq_mc, verbose=0)
            predictions_rescaled = scaler.inverse_transform(predictions)
            future_close_prices.append(predictions_rescaled[0, 0])
            predictions = predictions.reshape((1, 1, 1));  # Reshape predictions to (1, 1, 1)
            last_seq_mc = np.append(last_seq_mc[:, 1:, :], predictions, axis=1)
        future_close_prices_mc.append(future_close_prices)

    # Calculate the Monte Carlo average
    future_close_prices_mc_avg = np.mean(future_close_prices_mc, axis=0)

    # Plot forecasted prices
    plt.plot(close_df[seq_len:], label='Actual')
    #plt.plot(predictions_rescaled, label='Predicted')
    plt.plot(range(len(close_df[seq_len:]), len(close_df[seq_len:])+future_days), future_close_prices_mc_avg, label='Forecasted (MC Average)')
    plt.legend()
    plt.show()

    return future_close_prices_mc_avg

#ticker = 'FNGU'
#df = numeric_sentiment_pipeline(ticker)
#future_close_prices_mc_avg = gru_forcast(df, future_days=30, num_mc_simulations=100)
#print("Future High prices (MC Average):", future_close_prices_mc_avg)

### Sentiment and Numeric Forcasting Pipeline

In [ ]:
def forecast_pipeline(ticker, future_days):
  forcasted_df = pd.DataFrame()

  #data
  df = numeric_sentiment_pipeline(ticker)

  #forcasts
  forcasted_df['High']  = gru_forcast('High', df, future_days).round(2)
  forcasted_df['Low'] = gru_forcast('Low', df, future_days).round(2)
  forcasted_df['Close'] =  gru_forcast('Close', df, future_days).round(2)

  return forcasted_df

#ticker_list = ['SPY','FNGU']
#future_days=1
#predictions=[]
#for ticker in ticker_list:
    #prediction = forecast_pipeline(ticker, future_days)
    #send_forcast(ticker, prediction)
    #predictions.append(prediction)  # append each prediction to the list
    #predictions.append({'Ticker': ticker, 'Prediction': prediction})  # append each prediction to the list
    #print(prediction)

## Aggregation with telegram

In [ ]:

    forecasted_high = round(df['High'][0], 2)
    forecasted_low = round(df['Low'][0], 2)
    forecasted_close = round(df['Close'][0], 2)
    forecasted_high_vs_prev_close_difference = round(forecasted_high - prev_close, 2)

    message = f"Predicted Prices for: \n {ticker}:\n"
    message += f"High: {forecasted_high}\n"
    message += f"Low: {forecasted_low}\n"
    message += f"Close: {forecasted_close}\n"
    message += f"Forecasted High vs Prev Close Difference: {forecasted_high_vs_prev_close_difference}\n"

    url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage?chat_id={chat_id}&text={message}"

    requests.get(url).json()

#ticker = 'SPY'
#include_gpt=False
#future_days=1
#forcasts = forcast_pipeline(ticker, include_gpt, future_days)
#send_forcast(ticker, forcasts)

## Pipeline

In [15]:
import yfinance as yf
import pandas as pd

def active_forcaster_aggregator():
    ticker_list =  ['SPY','FNGU', 'AAPL', 'TSLA', 'CURLF', 'INQD','ACB','AUR','GRST',
                    'AXP', 'AMGN', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD',
                    #'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE',
                    #'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS','DOW'
                    ]

    future_days=1 #only works 1 day so far
    predictions = []  # create an empty list to store predictions
    for ticker in ticker_list:
        print(ticker)
        prediction = forecast_pipeline(ticker, future_days)
        stock_data = yf.download(ticker, period='1d')
        prev_close = round(stock_data['Close'].iloc[-1], 2)
        send_forcast(ticker, prediction, prev_close)
        predictions.append({'Ticker': ticker, 'High': prediction['High'].iloc[0], 'Low': prediction['Low'].iloc[0], 'Close': prediction['Close'].iloc[0]})

    forcasted_df = pd.DataFrame(predictions)

    # create new columns to store the previous day's prices and differences
    forcasted_df['Prev High Price']= None
    forcasted_df['Prev Low Price'] = None
    forcasted_df['Prev Close Price'] = None
    forcasted_df['High Price Difference'] = None
    forcasted_df['High Price Difference Percentage'] = None
    forcasted_df['Low Price Difference'] = None
    forcasted_df['Low Price Difference Percentage'] = None
    forcasted_df['Close Price Difference'] = None
    forcasted_df['Close Price Difference Percentage'] = None
    forcasted_df['Low to High Difference'] = None
    forcasted_df['Low to High Difference Percentage'] = None
    forcasted_df['Forecasted High vs Prev Close Difference'] = None
    forcasted_df['Forecasted High vs Prev Close Difference Percentage'] = None

    # iterate over each row in the DataFrame
    for index, row in forcasted_df.iterrows():
        ticker = row['Ticker']
        forecasted_high_price = round(float(row['High']), 2)
        forecasted_low_price = round(float(row['Low']), 2)
        forecasted_close_price = round(float(row['Close']), 2)

        # fetch the historical data for the ticker for the last available day
        stock_data = yf.download(ticker, period='1d')

        # extract the previous day's high, low, and close prices from the historical data
        prev_high_price = round(stock_data['High'].iloc[-1], 2)
        prev_low_price = round(stock_data['Low'].iloc[-1], 2)
        prev_close_price = round(stock_data['Close'].iloc[-1], 2)

        # update the 'Prev High Price', 'Prev Low Price', and 'Prev Close Price' columns
        forcasted_df.at[index, 'Prev High Price'] = prev_high_price
        forcasted_df.at[index, 'Prev Low Price'] = prev_low_price
        forcasted_df.at[index, 'Prev Close Price'] = prev_close_price

        # calculate the price differences and percentage differences
        high_price_difference = round(forecasted_high_price - prev_high_price, 2)
        high_price_difference_percentage = round((high_price_difference / prev_high_price) * 100, 2)
        low_price_difference = round(forecasted_low_price - prev_low_price, 2)
        low_price_difference_percentage = round((low_price_difference / prev_low_price) * 100, 2)
        close_price_difference = round(forecasted_close_price - prev_close_price, 2)
        close_price_difference_percentage = round((close_price_difference / prev_close_price) * 100, 2)

        low_to_high_difference = round(forecasted_high_price - prev_low_price, 2)
        low_to_high_difference_percentage = round((low_to_high_difference / prev_low_price) * 100, 2)

        # calculate the difference between the previous day's close price and the forecasted high price
        forecasted_high_vs_prev_close_difference = round(forecasted_high_price - prev_close_price, 2)
        forecasted_high_vs_prev_close_difference_percentage = round((forecasted_high_vs_prev_close_difference / prev_close_price) * 100, 2)

        #update the 'High Price Difference', 'High Price Difference Percentage', 'Low Price Difference', 'Low Price Difference Percentage', 'Close Price Difference', and 'Close Price Difference Percentage' columns
        forcasted_df.at[index, 'High Price Difference'] = high_price_difference
        forcasted_df.at[index, 'High Price Difference Percentage'] = high_price_difference_percentage
        forcasted_df.at[index, 'Low Price Difference'] = low_price_difference
        forcasted_df.at[index, 'Low Price Difference Percentage'] = low_price_difference_percentage
        forcasted_df.at[index, 'Close Price Difference'] = close_price_difference
        forcasted_df.at[index, 'Close Price Difference Percentage'] = close_price_difference_percentage
        forcasted_df.at[index, 'Low to High Difference'] = low_to_high_difference
        forcasted_df.at[index, 'Low to High Difference Percentage'] = low_to_high_difference_percentage
        forcasted_df.at[index, 'Forecasted High vs Prev Close Difference'] = forecasted_high_vs_prev_close_difference
        forcasted_df.at[index, 'Forecasted High vs Prev Close Difference Percentage'] = forecasted_high_vs_prev_close_difference_percentage

    # reset the index to a single index
    forcasted_df = forcasted_df.reset_index()

    forcasted_df.to_csv('forward_test_df.csv', index=False)
    return  forcasted_df

#forcasted = active_forcaster_aggregator()
#forcasted.head()

In [ ]:

message_1 = f"Good morning, please stand by for your daily forcast."
url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage?chat_id={chat_id}&text={message_1}"
requests.get(url).json()

forcasts = active_forcaster_aggregator()
forcasts.to_csv('forcasted_df.csv', index=False)


message_2 = f'Call if I can be of service. Happy hunting. S Rioghal mo dhream'
url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage?chat_id={chat_id}&text={message_2}"
requests.get(url).json()

forcasts#.head()

Output hidden; open in https://colab.research.google.com to view.

In [19]:
forcasts


,index,Ticker,High,Low,Close,Prev High Price,Prev Low Price,Prev Close Price,High Price Difference,High Price Difference Percentage,Low Price Difference,Low Price Difference Percentage,Close Price Difference,Close Price Difference Percentage,Low to High Difference,Low to High Difference Percentage,Forecasted High vs Prev Close Difference,Forecasted High vs Prev Close Difference Percentage
0,0,SPY,547.570007,544.280029,543.760010,547.19,541.49,544.44,0.38,0.07,2.79,0.52,-0.68,-0.12,6.08,1.12,3.13,0.57
1,1,FNGU,393.750000,384.239990,385.850006,393.7,376.0,384.87,0.05,0.01,8.24,2.19,0.98,0.25,17.75,4.72,8.88,2.31
2,2,AAPL,220.210007,217.789993,218.220001,219.49,216.01,217.96,0.72,0.33,1.78,0.82,0.26,0.12,4.2,1.94,2.25,1.03
3,3,TSLA,222.520004,220.509995,220.820007,222.28,215.33,219.8,0.24,0.11,5.18,2.41,1.02,0.46,7.19,3.34,2.72,1.24
4,4,CURLF,4.020000,4.010000,4.030000,4.08,3.94,4.03,-0.06,-1.47,0.07,1.78,0.0,0.0,0.08,2.03,-0.01,-0.25
5,5,INQD,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
6,6,ACB,5.960000,5.910000,5.940000,5.97,5.81,5.9,-0.01,-0.17,0.1,1.72,0.04,0.68,0.15,2.58,0.06,1.02
7,7,AUR,4.330000,4.120000,4.230000,4.34,4.21,4.23,-0.01,-0.23,-0.09,-2.14,0.0,0.0,0.12,2.85,0.1,2.36
8,8,GRST,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
9,9,AXP,246.649994,244.630005,245.369995,246.39,241.88,245.89,0.26,0.11,2.75,1.14,-0.52,-0.21,4.77,1.97,0.76,0.31


# Forward Testing



In [ ]:
import yfinance as yf
import pandas as pd

def forward_testing():
  # Later, to load the DataFrame from the CSV file
  forcasted_df_loaded = pd.read_csv('forcasted_df.csv')

  # create new columns to store the actual high, low, and close prices and dates
  forcasted_df_loaded['Actual High Price'] = None
  forcasted_df_loaded['Actual Low Price'] = None
  forcasted_df_loaded['Actual Close Price'] = None
  forcasted_df_loaded['Date'] = None
  forcasted_df_loaded['High Price Difference'] = None
  forcasted_df_loaded['High Price Difference Percentage'] = None
  forcasted_df_loaded['Low Price Difference'] = None
  forcasted_df_loaded['Low Price Difference Percentage'] = None
  forcasted_df_loaded['Close Price Difference'] = None
  forcasted_df_loaded['Close Price Difference Percentage'] = None

  # iterate over each row in the DataFrame
  for index, row in forcasted_df_loaded.iterrows():
      ticker = row['Ticker']
      forecasted_high_price = round(float(row['High']), 2)
      forecasted_low_price = round(float(row['Low']), 2)
      forecasted_close_price = round(float(row['Close']), 2)

      # fetch the historical data for the ticker for the last available day
      stock_data = yf.download(ticker, period='1d')

      # extract the actual high, low, and close prices and date from the historical data
      actual_high_price = round(stock_data['High'].iloc[-1], 2)
      actual_low_price = round(stock_data['Low'].iloc[-1], 2)
      actual_close_price = round(stock_data['Close'].iloc[-1], 2)
      high_price_date = stock_data.index[-1].date()

      # update the 'Actual High Price', 'Actual Low Price', 'Actual Close Price', and 'Date' columns
      forcasted_df_loaded.at[index, 'Actual High Price'] = actual_high_price
      forcasted_df_loaded.at[index, 'Actual Low Price'] = actual_low_price
      forcasted_df_loaded.at[index, 'Actual Close Price'] = actual_close_price
      forcasted_df_loaded.at[index, 'Date'] = high_price_date

      # calculate the price differences and percentage differences
      high_price_difference = round(actual_high_price - forecasted_high_price, 2)
      high_price_difference_percentage = round((high_price_difference / actual_high_price) * 100, 2)
      low_price_difference = round(actual_low_price - forecasted_low_price, 2)
      low_price_difference_percentage = round((low_price_difference / actual_low_price) * 100, 2)
      close_price_difference = round(actual_close_price - forecasted_close_price, 2)
      close_price_difference_percentage = round((close_price_difference / actual_close_price) * 100, 2)

      # update the 'High Price Difference', 'High Price Difference Percentage', 'Low Price Difference', 'Low Price Difference Percentage', 'Close Price Difference', and 'Close Price Difference Percentage' columns
      forcasted_df_loaded.at[index, 'High Price Difference'] = high_price_difference
      forcasted_df_loaded.at[index, 'High Price Difference Percentage'] = high_price_difference_percentage
      forcasted_df_loaded.at[index, 'Low Price Difference'] = low_price_difference
      forcasted_df_loaded.at[index, 'Low Price Difference Percentage'] = low_price_difference_percentage
      forcasted_df_loaded.at[index, 'Close Price Difference'] = close_price_difference
      forcasted_df_loaded.at[index, 'Close Price Difference Percentage'] = close_price_difference_percentage

  # set the 'Date' column as the index
  forcasted_df_loaded.set_index('Date', inplace=True)

  # reset the index to a single index
  forcasted_df_loaded = forcasted_df_loaded.reset_index()

  forcasted_df_loaded.to_csv('forward_test_df.csv', index=False)


  forcasted_df_loaded.to_csv('forward_test_df.csv', index=False)

## Uncomment and send it at the end of the day

In [ ]:
#forward_testing()
#forward_df = pd.read_csv('forward_test_df.csv')
#forward_df.head()